In [1]:
import nltk
nltk.download('vader_lexicon')
from nltk.sentiment import SentimentIntensityAnalyzer
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
import joblib
import pandas as pd
lr=joblib.load('lr.pkl')
vect=joblib.load('vector.pkl')
from tqdm import tqdm
#print("Torch version:", torch.__version__)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\dhrub\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
C:\Users\dhrub\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class RealDataSentimentAnalyzer:
    def __init__(self, tfidf_vectorizer, lr_model, finbert_model_path="ProsusAI/finbert",ticker='AAPL'):
        """
        Initialize all 3 sentiment analyzers.
        
        Parameters:
        - tfidf_vectorizer: From Week 2
        - lr_model: Logistic Regression from Week 2
        - finbert_model_path: FinBERT path
        """
        # VADER (Week 1)
        self.vader = SentimentIntensityAnalyzer()
        
        # Logistic Regression (Week 2)
        self.vectorizer = tfidf_vectorizer
        self.lr_model = lr_model
        
        # FinBERT (Week 3)
        self.tokenizer = AutoTokenizer.from_pretrained(finbert_model_path)
        self.model = AutoModelForSequenceClassification.from_pretrained(finbert_model_path)
        self.device = "cuda" if torch.cuda.is_available() else "cpu"
        self.model.to(self.device)
        self.df_news=None
        self.ticker=ticker
    
    def score_headline_vader(self, headline):
        """Score using VADER."""
        scores = self.vader.polarity_scores(headline)
        # Return compound score (-1 to +1)
        return scores['compound']
    
    def score_headline_lr(self, headline):
        """Score using Logistic Regression."""
        features = self.vectorizer.transform([headline])
        proba = self.lr_model.predict_proba(features)[0]
        # Return probability difference: positive - negative
        return proba[2] - proba[0]
    
    def score_headline_finbert(self, headline):
        """Score using FinBERT."""
        inputs = self.tokenizer(
            headline,
            return_tensors="pt",
            truncation=True,
            max_length=128
        ).to(self.device)
        
        with torch.no_grad():
            outputs = self.model(**inputs)
        
        probs = torch.softmax(outputs.logits, dim=-1)[0]
        # Return probability difference: positive - negative
        return (probs[2] - probs[0]).item()

    def score_all_headlines(self, headlines, batch_size=16):
        """
        Fast sentiment scoring with progress tracking.
        """
        headlines = list(headlines)
        total = len(headlines)
    
        # ---------- VADER (fast) ----------
        vader_scores = [
            self.vader.polarity_scores(h)['compound']
            for h in headlines
        ]
    
        # ---------- Logistic Regression (vectorized) ----------
        X = self.vectorizer.transform(headlines)
        lr_probas = self.lr_model.predict_proba(X)
        lr_scores = lr_probas[:, 2] - lr_probas[:, 0]
    
        # ---------- FinBERT (batched + progress) ----------
        finbert_scores = np.zeros(total)
    
        self.model.eval()
        with torch.no_grad():
            with tqdm(total=total, desc="FinBERT inference") as pbar:
                for i in range(0, total, batch_size):
                    batch = headlines[i:i + batch_size]
    
                    inputs = self.tokenizer(
                        batch,
                        return_tensors="pt",
                        padding=True,
                        truncation=True,
                        max_length=128
                    ).to(self.device)
    
                    outputs = self.model(**inputs)
                    probs = torch.softmax(outputs.logits, dim=-1)
    
                    scores = (probs[:, 2] - probs[:, 0]).cpu().numpy()
                    finbert_scores[i:i + len(scores)] = scores
    
                    # ✅ progress update
                    pbar.update(len(scores))
    
        # ---------- DataFrame ----------
        df = pd.DataFrame({
            "headline": headlines,
            "vader_sentiment": vader_scores,
            "lr_sentiment": lr_scores,
            "finbert_sentiment": finbert_scores
        })
    
        df["mean_sentiment"] = (
            df["vader_sentiment"]
            + df["lr_sentiment"]
            + df["finbert_sentiment"]
        ) / 3
    
        return df

    def load_financial_news(self, news_csv_path):
        """
        Load financial news dataset from CSV.
        Expected columns: date, headline, (optional) ticker
        """
        print(f"Loading news data from {news_csv_path}...")
        self.df_news = pd.read_csv(news_csv_path)

        # Convert date to datetime, coerce invalid dates to NaT
        self.df_news['date'] = pd.to_datetime(
            self.df_news['date'],
            errors='coerce'
        )

        # Remove rows with invalid date/time
        self.df_news = self.df_news.dropna(subset=['date'])

        # Extract date only
        self.df_news['date'] = self.df_news['date'].dt.date

        # Filter by ticker if column exists
        '''if 'ticker' in self.df_news.columns:
            self.df_news = self.df_news[self.df_news['ticker'] == self.ticker]'''

        self.df_news=self.df_news[['date','title']]

        print(f"Loaded {len(self.df_news)} news articles for {self.ticker}")
        return self.df_news

In [3]:
obj=RealDataSentimentAnalyzer(vect,lr)
obj.load_financial_news('apple_news_data.csv')
headlines=obj.df_news['title']
df=obj.score_all_headlines(headlines)
print(df)

Loading news data from apple_news_data.csv...
Loaded 29752 news articles for AAPL


FinBERT inference: 100%|█████████████████████████████████████████████████████████| 29752/29752 [19:34<00:00, 25.33it/s]

                                                headline  vader_sentiment  \
0      Berkshire Stock Hits Record Even as Company Re...           0.0000   
1                          What Is a Stock Market Index?           0.0000   
2      Could Investing $1,000 in Apple Make You a Mil...           0.0000   
3                           Dow Jones Industrial Average           0.0000   
4                             What Is the S&P 500 Index?           0.0000   
...                                                  ...              ...   
29747  Investor Expectations to Drive Momentum within...           0.0000   
29748  BioTelemetry, Inc. Enters Agreement to Provide...           0.4939   
29749  Factors of Influence in 2018, Key Indicators a...           0.7579   
29750  New Research: Key Drivers of Growth for Micros...           0.3818   
29751  Payment Data Systems Announces Apple Pay Suppo...           0.3182   

       lr_sentiment  finbert_sentiment  mean_sentiment  
0          0.02819

In [4]:
cols = ['vader_sentiment', 'lr_sentiment', 'finbert_sentiment']
corr_matrix = df[cols].corr()

print(corr_matrix)


                   vader_sentiment  lr_sentiment  finbert_sentiment
vader_sentiment           1.000000     -0.010022          -0.031852
lr_sentiment             -0.010022      1.000000           0.217220
finbert_sentiment        -0.031852      0.217220           1.000000


In [5]:
'''Hence in the entire dataset, vader has almost no correlation with lr and finbert. Lr and finbert have a slight correlation
which can be dut to a very small dataset used for training lr but finbert has been trained on millions of dataset and hence more reliable. Still its 
great to see even then lr has some suitable correlation with finbert even after been trained on a very small dataset'''

'Hence in the entire dataset, vader has almost no correlation with lr and finbert. Lr and finbert have a slight correlation\nwhich can be dut to a very small dataset used for training lr but finbert has been trained on millions of dataset and hence more reliable. Still its \ngreat to see even then lr has some suitable correlation with finbert even after been trained on a very small dataset'